<a href="https://colab.research.google.com/github/orizay/free-fonts/blob/master/02_%EB%8C%80%EB%A6%AC%EC%A0%90_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%EA%B0%80%EA%B3%B5%ED%95%98%EB%8A%94_%ED%85%8C%ED%81%AC%EB%8B%8910.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대리점에서 데이터를 가공하는 테크닉 10
- 테크닉 11_데이터를 읽어 들이자
- 테크닉 12_데이터의 오류를 살펴보자
- 테크닉 13_데이터에 오류가 있는 상태로 집계해 보자
- 테크닉 14_상품명의 오류를 수정하자
- 테크닉 15_금액의 결측치를 수정하자
- 테크닉 16_고객 이름의 오류를 수정하자
- 테크닉 17_날짜 오류를 수정하자
- 테크닉 18_고객 이름을 키로 두 개의 데이터를 결합(조인)하자
- 테크닉 19_정제한 데이터를 덤프하자
- 테크닉 20_ 데이를 집계하자

## 전제 조건
- 이 대리점에서는 상품 A~Z 까지 26개의 상품을 취급하며,매출 이력과 고객정보 데이터는 담사원이 시스템에 직접 입력합니다.
- 집계 기간에 상품 단가의 변동은 없었고 매출 이력은 시스템에서 CSV파일로 출력했습니다.
- 고객 정보는 대리점 관리자가 주별로 집계해서 엑셀로 관리합니다.
  - uriage.csv: 매출 이력, 기간은 2019년1월~2019년7월
  - kokyaku_daicho.xlsx: 대리점에서 관리하는 고객 정보

# **011/ 데이터를 읽어 들이자**

In [1]:
import pandas as pd

In [2]:
!git clone https://github.com/wikibook/pyda100

Cloning into 'pyda100'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 527 (delta 13), reused 19 (delta 8), pack-reused 489
Receiving objects: 100% (527/527), 477.16 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Checking out files: 100% (476/476), done.


In [3]:
uriage_data = pd.read_csv('/content/pyda100/2장/uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [4]:
kokyaku_data = pd.read_excel('/content/pyda100/2장/kokyaku_daicho.xlsx')
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


### 매출 이력의 데이터를 보면 item_name이나 item_price에 결측치나 오류가 있다.
- 이처럼 데이터에 나타나는 입력 오류나 표기 방법의 차이가 부정합을 일으킬 때'데이터의 정합성에 문제가 있다'라고 합니다.
- 데이터의 오류를 해소하고 정합성을 보장하는 것은 데이터 분석의 기초가 되는 중요한 부분입니다.
  - 정합성을 같추기 위해서는 먼저 데이터의 속성이나 의미를 이해해야 합니다.
  - 매출 이력에 purhase_date, item_name, itme_price, customer_name이 있는 것을 확인할 수 있습니다.
  - 정합성확보는 우선 데이터의 오류를 파악하는 것부터 시작합니다.

In [5]:
uriage_data['item_name'].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

### 매출 이력을 보면 '상품A', '상품a', '상 품 a' 처럼 공백이 포함되거나 알파벳 대소문자가 섞여 있는 것을 확인할 수 있습니다.
- 이대로 데이터 분석을 하면 각각 다른 상품으로 집계되여 원래 하니인'상품A'에 관해 정확한 집계 할 수 있습니다.

In [6]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

### 결측치 NaN을 확인할 수 있습니다. 
- 이런 결측치를 어떻게 처리하느냐가 향후 데이터 분석에 영향을 미칩니다.

# **013/ 데이터에 오류가 있는 상태로 집계해 보자**

### 데이터의 오류가 어느 정도 집계에 미치는지 확인함으로써 데이터의 정합성이 얼마나 중요한지 알 수 있습니다.

In [7]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,2,0,1,0,0,0,0,0,...,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,...,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,...,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0


### 데이터 수정 전의 집계 결과(상품별)
- 1~2행에서 날짜를 연월 형태로 변환합니다.
- 3행에서 세로축에 구입 연월, 가로축에 상품의 건수로 집계하고
- 4행에서 집계결과를 표시합니다
  - 이렇게 해서 데이터의 오류를 수정하지 않고 집계해 봤습니다
  - 결과를 보면,칼럼이 너무 많아 일부 생략됐지만,'상품s'처럼 동일한 상품이 다른 상품으로 집계된 것을 확인할 수 있습니다.

In [8]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,1500,1600,2000,0,0,4800,0,1900,0,0,0,0,0,...,6000,6500,7800,7000,10400,13500,16000,16500,19200,19500,21000,27000,20800,25500,36000,28500,28000,12600,41800,27600,24000,20000,0,0,300,0,0,0,0,0,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,0,0,0,0,0,0,0,0,400,0,0,0,0,900,0,0,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7600,5500,6600,7000,12800,8100,11000,14300,13200,16900,23800,25500,30400,30600,32400,32300,40000,37800,39600,43700,31200,20000,2600,0,0,0,0,0,0,1000,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6000,4000,11400,7700,14400,10800,13000,14300,21600,26000,16800,15000,32000,34000,27000,26600,22000,46200,35200,34500,38400,35000,0,0,0,400,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,2000,2200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1500,0,0,0,0,0,7500,0,0,0,0,1200,0,...,7200,8000,7800,9100,7200,14400,13000,12100,15600,14300,22400,18000,22400,28900,30600,28500,28000,16800,19800,20700,38400,32500,0,0,0,0,500,700,0,0,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,0,0,0,100,0,1900,100,0,0,0,0,0,0,0,1100,1300,0,0,0,0,0,2200,0,0,0,0,0,0,600,0,...,5200,9000,9000,14000,12000,13500,8000,15400,12000,20800,23800,24000,28800,15300,32400,20900,26000,33600,8800,36800,31200,45000,0,0,300,0,0,0,0,0,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,0,0,0,0,0,0,0,300,0,0,0,700,0,0,0,0,0,1500,0,0,2100,0,0,0,0,100,0,0,0,800,...,7200,6500,9600,7700,8800,12600,20000,15400,14400,19500,15400,19500,24000,23800,27000,34200,22000,29400,33000,25300,24000,30000,0,0,0,0,0,0,900,0,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1700,0,0,0,0,500,0,0,800,0,0,0,0,1500,0,0,0,2200,0,0,0,0,0,500,0,0,...,6000,4500,9600,11900,12800,13500,17000,9900,20400,13000,18200,25500,38400,18700,25200,38000,34000,23100,52800,32200,38400,27500,0,0,0,0,0,0,900,0,0,0,1500,0,1800,0,0,0,0,0


### 데이터 오류 수정 전 집계 결과(금액)
- 이미 날짜 처리는 끝냈기 때문에 pivot_table로 집계 처리만 하면 됩니다.
- 마찬가지로 집계 결과가 올바르지 않은 것을 확인할 수 있습니다

  - 이렇게 데이터에 오류가 있는 상태로 집계 및 분석을 실시하면 전혀 의미 없는 결과가 나오기 때문에 데이터 가공이 분석의 전처리로 얼마나 중요한지 확인할 수 있습니다.

# **014/ 상품명 오류 수정하자**

### 우선 상품명의 오류를 수정해 봅시다. 이번 사례는 비교적 간다한 오류여서 '공백' 오류를 수정하는 것만으로 해결할 수 있을 것 같습니다.

In [9]:
print(len(pd.unique(uriage_data.item_name)))

99


In [22]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
uriage_data.sort_values(by=['item_name'], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


### 처리 결과
- 1행에서 상품명에 있는 소문자를 str.upper()를 이용해 대문자로 변환합니다
- 2, 3행의 str.replace()로 공백을 제거 합니다
- 4행에서 데이터를 itme_name 순으로 정렬해 화면에 표시합니다
  - 표를 스크롤해서 데이터를 보면 바르게 수정된 것처럼 보이지만, 반드시 결과를 검증하는 것을 잊어서는 안 됩니다.

In [23]:
print(len(pd.unique(uriage_data['item_name'])))
print(pd.unique(uriage_data['item_name']))

26
['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']


# **015/ 금액의 결측치를 수정하자**

### 금액의 결측치를 수정해 봅니다
- 결측치에 따라 현장에서 조사를 거쳐 수정학나 부서원이 결측치를 채워 넣는 등의 여러 가지 대응이 가능합니다
- 이번 경우에는 상품 단가가 집계 중에 변하지 않았기 때문에 프로르램으로 결측치를 수정할 수 있습니다

In [24]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

### 결과를 보면 item_price 금액에 결측치가 있는 것을 확인할수 있습니다.




In [25]:
flg_is_null = uriage_data['item_price'].isnull()
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
    uriage_data['item_price'].loc[(fig_is_null) & (uriage_data['item_name'] == trg)] = price

uriage_data.head()

NameError: ignored

### 결측치를 처리한 결과
- 1행에서 item_price 중에 결측치가 있는 곳을 조사합니다. 이 처리는 flg_is_null변수에 어떤행에 결측치가 있는지를 저장합니다
- 2행의 반복문 처리에는 list(uriage_data.loc[flg_is_null, 'item_name'].unique())를 이용합니다.
  - 앞에서 생성한 flg_is_null을 이용해서 결측치가 있는 상품명 리스트를 작성합니다.한해에 여러가지 처리가 섞여 있으므로 나눠서 설명합니다
  - list()는 변수의 값을 리스트 형식으로 변환합니다. 그리고 uriage_data.loc[glg_is_null, 'item_name']에서 loc 함수는 조건에 일치하는 데이터를 추출합니다.
  - 이번에 사용할 조건은 '금액에 결측치가 있다'이기 때문에 생성한 flg_is_null을 이용해 조건을 지정합니다.
  - 2번째 item_name은 조건과 일치하는 데이터 중에서 어떤 칼럼을 가져올지를 지정합니다.이렇게 결측치가 존재하는 상품명을 추출합니다.
  - 마지막 unique()는 추출한 상품명에서 중복을 제거 합니다.불필요한 반복을 없애기 위해서입니다.
- 반복문 속의 price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()를 설명합니다.
  - 이것은 '결측치가 있는 상품'가 같은 상품이며 금액이 올바르게 입력된 행을 loc로 찾고, 그 금액을 가져옵니다.
  - loc()의 조건으로는 (~flg_is_null) & (uriage_data['item_name'] == trg)와 같이 여러 가지를 지정할 수 있습니다.
  - ~flg_is_null에서 ~는 부정 연산자라고 하며, flg_is_null == False와 같습니다.
  - 이렇게 결손치가 있는 상품과 같은 상품의 다른 데이터에서 금액을 가져올 수 있습니다.
- 가져온 금액으로 데이터를 수정합니다.
  - uriage_data['item_price'].loc[(flg_is_null) & (uriage_data['item_name'] == trg)] == price 에서는 매출 이력의 item_price칼럼에서 loc()로 결측치가 있는 데이터를 추출하고, 앞에서 가져온 데이터 price를 결측치에 대입합니다.
  - 반복문 처리가 끝나면 수정 후 매출 이력 처음 5행을 표시합니다

In [20]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [21]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + '의 최고가:' + str(uriage_data.loc[uriage_data['item_name'] == trg]
                               ['item_price'].max()) + '의 최저가:' + str(uriage_data.loc[uriage_data['item_name'] == trg]
                               ['item_price'].min(skipna=False)))

상품A의 최고가:100.0의 최저가:nan
상품B의 최고가:200.0의 최저가:nan
상품C의 최고가:300.0의 최저가:nan
상품D의 최고가:400.0의 최저가:nan
상품E의 최고가:500.0의 최저가:nan
상품F의 최고가:600.0의 최저가:nan
상품G의 최고가:700.0의 최저가:nan
상품H의 최고가:800.0의 최저가:nan
상품I의 최고가:900.0의 최저가:nan
상품J의 최고가:1000.0의 최저가:nan
상품K의 최고가:1100.0의 최저가:nan
상품L의 최고가:1200.0의 최저가:nan
상품M의 최고가:1300.0의 최저가:nan
상품N의 최고가:1400.0의 최저가:nan
상품O의 최고가:1500.0의 최저가:nan
상품P의 최고가:1600.0의 최저가:nan
상품Q의 최고가:1700.0의 최저가:nan
상품R의 최고가:1800.0의 최저가:nan
상품S의 최고가:1900.0의 최저가:nan
상품T의 최고가:2000.0의 최저가:nan
상품U의 최고가:2100.0의 최저가:nan
상품V의 최고가:2200.0의 최저가:nan
상품W의 최고가:2300.0의 최저가:nan
상품X의 최고가:2400.0의 최저가:nan
상품Y의 최고가:2500.0의 최저가:nan
상품Z의 최고가:2600.0의 최저가:2600.0


### 결측치 체크 결과(금액 수정 검증)
- 1행에서 모든 상품에 대해서 반복문 처리를 합니다.반복문에서는 상품에 설정된 최대 금액과 최소 금액을 출력해 봅니다.
- 결과에서 모든 상품의 최대 금액과 최소 금액이 일치하는 것으로 봐서 성공적으로 금액을 수정했자는 것을 알 수 있습니다.
- min(skipna=False)에서 skipna는 NaN의 무시 여부를 설정합니다. 이번에는 False로 지정했기 때문에 NaN이 존재할 경우 최솟값으로 NaN으로 표시됩니다.

# **016/ 고객 이름의 오류를 수정하자**

In [18]:
kokyaku_data['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

### 고객 정보의 고객이름

In [19]:
uriage_data['customer_name'].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

### 매출 이력의 고객 이름

### 고객 정보와 이력의 고객 이름을 비교해 보면 고객 정보의 고객 이름에는 성과 이름 사이에 공백이 있지만, 매출 이력의 고객 이름에는 공백이 없습니다.좀 더 자세히 보면,고객 정보의 고객 이름에는 공백이 한 칸이나 두 칸,아니면 공백이 없는 등 서식이 혼재 되어 있습니다.
- 이대로 매출 이력과 고객 정보를 결합하면 정상적으로 결합되지 않습니다.
- 고객이름 이외에는 두 개의 데이터를 연결할 키가 존재하지 않기 때문에 고객 이름을 반드시 수정해야 합니다.

In [26]:
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace('  ', '')
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace(' ','')
kokyaku_data['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

### 이번에는 테스트 데이터이기 때문에 아주 간단한 오류를 다루지만, 실제 데이터의 이름에는 좀 더 복잡한 오류가 있을 수 있습니다.
- 이름 입력 오류의 경우에는 이것이 잘못 입력한 건지 아니면 다른 사람인지를 판단할 수 없기 때문에 단순히 프로그램으로 수정할 수 없습니다.
- 담당자에게 물어서 작업하는 수밖에 없습니다.

# **017/ 날짜 오류를 수정하자**

### 프로그램밍 언어에 따라 오류를 자동으로 수정해주는 경우도 있지만,이번 경우에는 yyyy년 mm월 dd일의 데이터가 바르게 날짜로 인식되고 있지 않습니다.

In [28]:
flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

22

### 숫자로 된 장소 확인
- 1행의 flg_is_serial = kokyaku_data['등록일'].astype('str').srr.isdigit()에서 고객 정보의 등록일이 숫자인지 아닌지를 str.isdigit()로 판정합니다.
- 판정 결과 , 숫자로 된 장소를 flg_is_serial에 저장합니다.
- 2행은 나중에 검증을 하기 위해서 개수를 표시합니다.

In [30]:
fromSerail = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '등록일'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
fromSerail

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

### 숫자를 날짜로 변경
- pd.to_timedelta() 함수를 이용해서 숫자를 날짜로 변환합니다.
- loc()를 이용해서 flg_is_serial 조건으로 데이터를 추출하고 날짜로 변경합니다.
- 숫자가 날짜형으로 변환된 것을 확인할 수 있습니다.

In [31]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '등록일'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

### 날짜 변환 결과
- 2018/01/04 와 같이 슬래시로 구분된 서식을 하이픈으로 구분된 서식으로 통일하기 위한 처리를 했습니다.
- 숫자를 날짜로 수정한 데이터와 서식을 변경한 데이터를 결합해서 데이터를 갱신합니다.

In [32]:
kokyaku_data['등록일'] = pd.concat([fromSerail, fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


### 날짜 갱신 결과
- 숫자를 날짜로 수정한 데이터 fromSerial과 서식을 변경한 데이터 fromString을 concat으로 결합하고,원래'등록일'에 대입해 갱신합니다.
  - 결과를 보면 ,등록일이 깨끗하게 수정된 것을 알수 있습니다.

In [35]:
kokyaku_data['등록연월'] = kokyaku_data['등록일'].dt.strftime('%Y%m')
rslt = kokyaku_data.groupby('등록연월').count()['고객이름']
print(rslt)
print(len(kokyaku_data))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


### 등록월 집계 결과
- 수정한 날짜에서'등록 연월'을 작성하고 groupby()로 집계합니다.
- groupby 를 한후, count()로 데이터이 개수를 집계하고 집계 결과를 화면에 출력합니다.
- 검증을 위해 len(kokyaku_data)로 고객 대장의 총데이터 수를 표시합니다.

In [36]:
flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

0

### 숫자 항목 유무
- 처음에 22개였던 결과가 0개가 도어 모든 숫자 데이터가 날짤로 수정된 것을 확인할 수 있습니다.

# **018/ 고객 이름을 키로 두 개의 데이터를 결합(조인)하자**

In [37]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='고객이름', how='left')
join_data = join_data.drop('customer_name', axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,NaN,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,NaN,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,NaN,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,NaN,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,NaN,201907,김승주,E시,2018-07-15,201807


### 결합 결과
- left_on, right_on 으로 결합할 데이터를 지정합니다.
- left_on의 인수로는 uriage_data의 customer_name칼럼을 키로 지정합니다.
- right_on의 인수로는 kokyaku_data의 고객 이름 칼럼을 키로 지정합니다.
- how는 결합 밥버으로 , left로 지정 했습니다.
  - 이것은 uriage_data를 기준으로 kokyaku_data를 결합한다는 의미 입니다.

- 데이터를 가공함으로써 분석에 적합한 데이터의 형태가 되었습니다.이런 데이터 가공을 '데이터 정제'하고도 합니다.

# **019/ 정제한 데이터를 덤프하자**

### 데이터를 수정해서 깔끔하게 정리했짐만,여기서 중단하면 지금까지의 모든 처리를 다시 해야 합니다.우리의 목적은 '데이터 가공 및 정제'가 이니고 '데이터 분석'입니다. 깨끗해진 데이터를 파일로 출력(덤프)해두고, 분석할 때 출력한 파일을 다시 읽어 들이면 데이터 정제를 다시 할 필요가 없습니다.

In [40]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '고객이름', '지역', '등록일']]
dump_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,NaN,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,NaN,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,NaN,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,NaN,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,NaN,김승주,E시,2018-07-15


### 수정 결과
- join_data에서 필요한 칼럼을 임의의 순서로 정렬합니다.
- 캄럼명만 지정하면 되기 때문에 간단합니다.

In [41]:
dump_data.to_csv('dump_data.csv', index=False)

# **020/ 데이터를 집계하자**

In [42]:
import_data = pd.read_csv('dump_data.csv')
import_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,201907,상품S,NaN,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,201905,상품A,NaN,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,201904,상품A,NaN,김강현,D시,2017-06-07 00:00:00
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01 00:00:00
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31 00:00:00
2996,2019-03-29 11:14:00,201903,상품Q,NaN,김지율,B시,2017-03-15 00:00:00
2997,2019-07-14 12:56:00,201907,상품H,NaN,김승주,E시,2018-07-15 00:00:00


In [52]:
# purchase_month를 세로 축으로 해서 상품별로  집계해 봅니다.
byItem = import_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size',fill_value=0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,20,19,19,16,24,16,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,16,11,15,20,19,19,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,14,20,22,13,11,21,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,14,15,11,20,15,15,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,18,13,18,19,18,20,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,15,15,17,16,15,18,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,12,17,11,15,22,26,15,19,23,21,13,28,16,18,12,0


### 26columns 상품 A~Z 까지 26개의 상품이 구입 연월별로 집계된 것을 알 수 있습니다

In [56]:
# purchase_month를 세로축으로 지정하고,매출 금액, 고객,지역을 집계해 봅시다
byPrice = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
byPrice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1500,1600,5400,6000,6500,7800,7000,11200,14400,16000,17600,20400,20800,22400,31500,22400,25500,36000,32300,30000,12600,44000,27600,28800,20000,0
201902,1700,2400,6000,8000,5500,6600,7700,12800,9000,12000,14300,13200,16900,25200,27000,30400,30600,34200,34200,42000,37800,41800,43700,36000,20000,2600
201903,1300,4200,5100,6400,4500,12000,7700,14400,10800,14000,14300,21600,27300,16800,15000,33600,34000,27000,26600,22000,46200,35200,34500,38400,35000,0
201904,1400,2200,6000,7200,8500,9000,9800,7200,14400,13000,12100,15600,14300,22400,19500,22400,30600,30600,30400,30000,16800,22000,23000,38400,40000,0
201905,2100,2600,4200,5200,9000,9600,14000,12000,13500,9000,17600,13200,22100,23800,24000,28800,17000,32400,24700,28000,33600,11000,39100,31200,47500,0
201906,2100,2400,2700,7200,6500,9600,9100,9600,13500,20000,15400,14400,19500,15400,21000,25600,23800,27000,34200,22000,31500,33000,27600,26400,30000,0
201907,1600,3600,4500,6000,5500,9600,11900,13600,14400,17000,9900,20400,13000,18200,28500,38400,20400,27000,38000,36000,23100,57200,32200,38400,27500,0


In [58]:
byCustomer = import_data.pivot_table(index='purchase_month', columns='고객이름', aggfunc='size',fill_value=0)
byCustomer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,김도경,김도영,김도원,김도윤,김도율,김도하,김도현,김도훈,김동건,김동우,김동욱,김동하,김동현,김라온,김민건,김민규,김민기,김민서,김민석,김민수,김민우,김민재,김민준,김민찬,김민혁,김민호,김범준,김상민,김상현,김서윤,...,김지혁,김지호,김지환,김지후,김지훈,김진우,김찬영,김찬우,김태경,김태민,김태양,김태영,김태우,김태윤,김태인,김태준,김태현,김태훈,김하람,김하랑,김하민,김하윤,김하율,김하준,김하진,김한결,김현민,김현서,김현석,김현성,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,4,0,0,2,2,2,0,3,2,2,2,1,3,3,3,1,3,1,4,2,0,3,3,0,4,4,5,1,2,2,...,3,5,2,2,2,1,6,5,1,2,1,2,3,4,0,0,0,1,1,2,3,3,2,1,7,1,0,1,3,2,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,1,6,4,4,1,2,4,3,1,2,0,1,1,3,1,1,3,2,2,3,3,3,9,0,5,2,2,0,1,2,...,3,0,1,1,1,1,1,3,4,1,1,0,1,2,2,0,0,1,2,3,3,2,1,0,2,0,4,4,5,3,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,3,0,3,3,0,2,4,2,5,1,1,0,3,1,2,1,5,5,1,0,1,5,1,1,1,1,3,3,0,6,...,4,4,2,1,1,3,0,3,2,2,3,4,3,2,4,2,4,3,1,2,2,1,2,1,1,4,0,5,0,2,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,1,3,1,3,1,4,2,2,1,4,4,1,1,3,2,2,4,2,0,1,2,1,0,2,5,4,2,3,2,2,...,1,4,3,1,0,4,1,1,1,5,4,1,0,0,2,1,3,5,0,5,3,4,4,4,1,0,1,4,4,3,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,3,2,3,2,2,1,1,1,2,2,0,2,4,4,0,1,2,2,2,1,3,1,3,5,2,1,2,1,2,2,...,3,2,6,3,4,0,0,2,2,4,2,5,1,0,3,1,1,1,1,5,4,4,2,1,1,2,2,2,0,2,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,2,2,5,4,2,1,1,2,1,3,1,2,2,3,3,4,3,3,6,2,1,2,1,2,0,2,1,3,4,4,...,0,1,3,1,1,2,2,1,4,4,1,3,2,3,2,3,1,2,2,0,2,3,2,3,0,1,1,1,1,4,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,4,2,1,1,3,2,3,1,1,3,5,2,1,2,2,1,1,2,2,1,5,0,3,4,2,2,5,4,4,2,...,2,0,1,2,5,2,2,4,6,1,1,5,2,2,4,1,2,2,2,2,2,2,2,4,1,0,1,3,2,4,2,1,0,4,0,2,6,1,2,3


In [59]:
byRegion = import_data.pivot_table(index='purchase_month', columns='지역', aggfunc='size', fill_value=0)
byRegion

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [61]:
# 집계 기간에 구매 이력이 없는 사용자를 확인해 봅니다.
away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='고객이름', how='right')
away_data[away_data['purchase_date'].isnull()][['고객이름','등록일']]

,고객이름,등록일
2999,김서우,2019-04-23


### 데이터를 분석함에 있어 가장 중요한 데이터 가공의 기초를 배웠습니다.
- 데이터의 상태를 살펴보고 어떻게 가공할 것인가를 현장 멤버들과 상의하면서 정혹하게 데이터를 가공해야 합니다.
- 이 부분을 소홀히 하면 나중에 큰코다칠 수 있으니 이번처럼 기초적인 정제뿐만아니라,복잡한 데이터의 정제도 해 보기를 바랍니다.